<h1 align=center>Capítulo 6</h1>
<h2 align=center>Codificação e Processamento de Dados</h2>
<p align=center><img src=https://energiainteligenteufjf.com.br/wp-content/uploads/2021/02/curso-insofti-introducao-ao-processamento-de-dados-ipd.jpg width=500></p>

O foco principal deste capítulo é usar o Python para processar dados apresentados em diferentes tipos de codificações comuns, como arquivos CSV, JSON, XML e registros compactados em binários. Ao contrário do capítulo sobre estruturas de dados, este capítulo não se concentra em algoritmos específicos, mas no problema de obter e retirar dados de um programa.

## 6.1. Lendo e gravando dados CSV

**Problema**

Você deseja ler ou gravar dados codificados como um arquivo CSV.

**Solução**

Para a maioria dos tipos de dados CSV, use a biblioteca `csv`. Por exemplo, suponha que você tenha alguns dados do mercado de ações em um arquivo chamado *stocks.csv* assim:
~~~python
 Symbol,Price,Date,Time,Change,Volume
 "AA",39.48,"6/11/2007","9:36am",-0.18,181800
 "AIG",71.38,"6/11/2007","9:36am",-0.15,195500
 "AXP",62.58,"6/11/2007","9:36am",-0.46,935000
 "BA",98.31,"6/11/2007","9:36am",+0.12,104800
 "C",53.08,"6/11/2007","9:36am",-0.25,360900
 "CAT",78.29,"6/11/2007","9:36am",-0.23,225400
~~~


In [1]:
import csv
with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        print(row)

[]
['AA', '39.48', '6/11/2007', '9:36am', '-0.18', '181800']
[]
['AIG', '71.38', '6/11/2007', '9:36am', '-0.15', '195500']
[]
['AXP', '62.58', '6/11/2007', '9:36am', '-0.46', '935000']
[]


No código anterior, *row* será uma tupla. Assim, para acessar determinados campos, você precisará usar indexação, como row[0] (Símbolo) e row[4] (Alterar).
Como essa indexação geralmente pode ser confusa, este é um lugar onde você pode querer considerar o uso de tuplas nomeadas. Por exemplo:
~~~python
from collections import namedtuple
with open('stock.csv') as f:
 f_csv = csv.reader(f)
 headings = next(f_csv)
 Row = namedtuple('Row', headings)
 for r in f_csv:
 row = Row(*r)
 # Process row
 ...
~~~


Isso permitiria que você usasse os cabeçalhos das colunas, como `row.Symbol` e `row.Change`, em vez de índices. Deve-se notar que isso só funciona se os cabeçalhos das colunas forem identificadores Python válidos. Caso contrário, talvez seja necessário massagear os títulos iniciais (por exemplo, substituindo caracteres não identificadores por sublinhados ou similares).

Outra alternativa é ler os dados como uma sequência de dicionários. Para isso, use este código:

In [2]:
print('Reading as dicts')
with open('stocks.csv') as f:
    f_csv = csv.DictReader(f)
    for row in f_csv:
        # process row
        print('    ', row)

Reading as dicts
     {'Symbol': 'AA', 'Price': '39.48', 'Date': '6/11/2007', 'Time': '9:36am', 'Change': '-0.18', 'Volume': '181800'}
     {'Symbol': 'AIG', 'Price': '71.38', 'Date': '6/11/2007', 'Time': '9:36am', 'Change': '-0.15', 'Volume': '195500'}
     {'Symbol': 'AXP', 'Price': '62.58', 'Date': '6/11/2007', 'Time': '9:36am', 'Change': '-0.46', 'Volume': '935000'}


Nesta versão, você acessaria os elementos de cada linha usando os cabeçalhos de linha. Por exemplo, row['Symbol'] ou row['Change']. 

Para gravar dados CSV, você também usa o módulo `csv`, mas cria um objeto de gravação. Por exemplo:

In [3]:
headers = ['Symbol','Price','Date','Time','Change','Volume']
rows = [('AA', 39.48, '6/11/2007', '9:36am', -0.18, 181800),
        ('AIG', 71.38, '6/11/2007', '9:36am', -0.15, 195500),
        ('AXP', 62.58, '6/11/2007', '9:36am', -0.46, 935000),
       ]
with open('stocks.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)

Se você tiver os dados como uma sequência de dicionários, faça o seguinte:

In [4]:
headers = ['Symbol', 'Price', 'Date', 'Time', 'Change', 'Volume']
rows = [{'Symbol':'AA', 'Price':39.48, 'Date':'6/11/2007','Time':'9:36am', 'Change':-0.18, 'Volume':181800},
        {'Symbol':'AIG', 'Price': 71.38, 'Date':'6/11/2007','Time':'9:36am', 'Change':-0.15, 'Volume': 195500},
        {'Symbol':'AXP', 'Price': 62.58, 'Date':'6/11/2007','Time':'9:36am', 'Change':-0.46, 'Volume': 935000},
       ]

with open('stocks.csv','w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)

**Discussão**

Você quase sempre deve preferir o uso do módulo `csv` em vez de tentar manualmente dividir e analisar os dados CSV. Por exemplo, você pode estar inclinado a apenas escrever algum código como este:

In [5]:
with open('stocks.csv') as f:
    for line in f:
        row = line.split(',')
        print(row)
        # process row


['Symbol', 'Price', 'Date', 'Time', 'Change', 'Volume\n']
['\n']
['AA', '39.48', '6/11/2007', '9:36am', '-0.18', '181800\n']
['\n']
['AIG', '71.38', '6/11/2007', '9:36am', '-0.15', '195500\n']
['\n']
['AXP', '62.58', '6/11/2007', '9:36am', '-0.46', '935000\n']
['\n']


O problema com essa abordagem é que você ainda precisará lidar com alguns detalhes desagradáveis. Por exemplo, se algum dos campos estiver entre aspas, você terá que remover as aspas. Além disso, se um campo entre aspas contiver uma vírgula, o código será interrompido produzindo uma linha com o tamanho errado.

Por padrão, a biblioteca `csv` é programada para entender as regras de codificação CSV usadas pelo Microsoft Excel. Esta é provavelmente a variante mais comum e provavelmente lhe dará a melhor compatibilidade. No entanto, se você consultar a documentação do `csv`, verá algumas maneiras de ajustar a codificação para diferentes formatos (por exemplo, alterando o caractere separador etc.). Por exemplo, se você quiser ler dados delimitados por tabulação, use isto:

In [6]:
# Example of reading tab-separated values
with open('stocks.tsv') as f:
    f_tsv = csv.reader(f, delimiter='\t')
    for row in f_tsv:
        # Process row
        print(row)


['Symbol', 'Price', 'Date', 'Time', 'Change', 'Volume']
['AA', '39.48', '6/11/2007', '9:36am', '-0.18', '181800']
['AIG', '71.38', '6/11/2007', '9:36am', '-0.15', '195500']
['AXP', '62.58', '6/11/2007', '9:36am', '-0.46', '935000']
['BA', '98.31', '6/11/2007', '9:36am', '+0.12', '104800']
['C', '53.08', '6/11/2007', '9:36am', '-0.25', '360900']
['CAT', '78.29', '6/11/2007', '9:36am', '-0.23', '225400']


Se você estiver lendo dados CSV e convertendo-os em tuplas nomeadas, precisará ter um pouco de cuidado ao validar os cabeçalhos das colunas. Por exemplo, um arquivo CSV pode ter uma linha de cabeçalho contendo caracteres identificadores inválidos como este:
~~~python
Street Address,Num-Premises,Latitude,Longitude
5412 N CLARK,10,41.980262,-87.668452
~~~

Isso fará com que a criação de uma tupla nomeada falhe com uma exceção `ValueError`. Para contornar isso, talvez seja necessário esfregar os cabeçalhos primeiro. Por exemplo, carregando uma substituição regex em caracteres identificadores inválidos como este:

In [7]:
import re
from collections import namedtuple
with open('stocks.tsv') as f:
    f_csv = csv.reader(f, delimiter='\t')
    headers = [ re.sub('[^a-zA-Z_]', '_', h) for h in next(f_csv) ]
    Row = namedtuple('Row', headers)
    for r in f_csv:
        row = Row(*r)

Também é importante enfatizar que o `csv` não tenta interpretar os dados ou convertê-los para um tipo diferente de uma *string*. Se essas conversões forem importantes, isso é algo que você precisará fazer sozinho. Aqui está um exemplo de como realizar conversões de tipo extra em dados CSV:

In [8]:
col_types = [str, float, str, str, float, int]
with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        # Apply conversions to the row items
        row = tuple(convert(value) for convert, value in zip(col_types, row))
        print(row)

()
('AA', 39.48, '6/11/2007', '9:36am', -0.18, 181800)
()
('AIG', 71.38, '6/11/2007', '9:36am', -0.15, 195500)
()
('AXP', 62.58, '6/11/2007', '9:36am', -0.46, 935000)
()


Alternativamente, aqui está um exemplo de conversão de campos selecionados de dicionários:

In [9]:
print('Reading as dicts with type conversion')
field_types = [ ('Price', float),('Change', float), ('Volume', int)]
with open('stocks.csv') as f:
    for row in csv.DictReader(f):
        row.update((key, conversion(row[key]))
                   for key, conversion in field_types)
        print(row)

Reading as dicts with type conversion
{'Symbol': 'AA', 'Price': 39.48, 'Date': '6/11/2007', 'Time': '9:36am', 'Change': -0.18, 'Volume': 181800}
{'Symbol': 'AIG', 'Price': 71.38, 'Date': '6/11/2007', 'Time': '9:36am', 'Change': -0.15, 'Volume': 195500}
{'Symbol': 'AXP', 'Price': 62.58, 'Date': '6/11/2007', 'Time': '9:36am', 'Change': -0.46, 'Volume': 935000}


Em geral, você provavelmente vai querer ter um pouco de cuidado com essas conversões. No mundo real, é comum que os arquivos CSV tenham valores ausentes, dados corrompidos e outros problemas que interromperiam as conversões de tipo. Portanto, a menos que seus dados sejam garantidos como livres de erros, isso é algo que você precisará considerar (talvez seja necessário adicionar tratamento de exceção adequado).

Por fim, se seu objetivo ao ler dados CSV é realizar análises e estatísticas de dados, convém examinar o pacote Pandas. Pandas inclui uma função conveniente `pandas.read_csv()` que carregará dados CSV em um objeto DataFrame. A partir daí, você pode gerar várias estatísticas resumidas, filtrar os dados e realizar outros tipos de operações de alto nível. Um exemplo é dado na Receita 6.13.

## 6.2. Lendo e gravando dados JSON

**Problema**

Você deseja ler ou gravar dados codificados como JSON (JavaScript Object Notation).

**Solução**

O módulo `json` fornece uma maneira fácil de codificar e decodificar dados em JSON. As duas funções principais são `json.dumps()` e `json.loads()`, espelhando a interface usada em outras bibliotecas de serialização, como `pickle`. Veja como você transforma uma estrutura de dados Python em JSON:

In [10]:
import json
data = {
    'name' : 'ACME',
    'shares' : 100,
    'price' : 542.23
}

json_str = json.dumps(data)

Veja como você transforma uma string codificada em JSON de volta em uma estrutura de dados Python:

In [11]:
data = json.loads(json_str)

Se estiver trabalhando com arquivos em vez de strings, você pode usar alternativamente `json.dump()` e `json.load()` para codificar e decodificar dados JSON. Por exemplo:

In [12]:
# Writing JSON data
with open('data.json', 'w') as f:
    json.dump(data, f)

In [13]:
# Reading data back
with open('data.json', 'r') as f:
    data = json.load(f)

**Discussão**

A codificação JSON oferece suporte aos tipos básicos de **None**, **bool**, **int**, **float** e **str**, bem como listas, tuplas e dicionários que contêm esses tipos. Para dicionários, as chaves são consideradas strings (qualquer chave não string em um dicionário é convertida em strings durante a codificação).Para estar em conformidade com a especificação JSON, você deve codificar apenas listas e dicionários do Python. Além disso, em aplicativos da Web, é prática padrão que o objeto de nível superior seja um dicionário.

O formato da codificação JSON é quase idêntico à sintaxe do Python, exceto por algumas pequenas alterações. Por exemplo, `True` é mapeado para **true**, `False` é mapeado para **false** e `None` é mapeado para **null**. Aqui está um exemplo que mostra como a codificação se parece:

In [14]:
json.dumps(False)

'false'

In [15]:
d = {'a': True,
     'b': 'Hello',
     'c': None}
json.dumps(d)

'{"a": true, "b": "Hello", "c": null}'

Se você estiver tentando examinar os dados que decodificou do JSON, muitas vezes pode ser difícil determinar sua estrutura simplesmente imprimindo-os, especialmente se os dados contiverem um nível profundo de estruturas aninhadas ou muitos campos. Para ajudar com isso, considere usar a função `pprint()` no módulo `pprint`. Isso alfabetizará as chaves e produzirá um dicionário de uma maneira mais sã. Aqui está um exemplo que ilustra como você imprimiria os resultados de uma pesquisa no Twitter:

In [16]:
from urllib.request import urlopen
import json
u = urlopen('https://nominatim.openstreetmap.org/search.php?q=Manaus&format=jsonv2')
resp = json.loads(u.read().decode('utf-8'))
from pprint import pprint
pprint(resp)

[{'boundingbox': ['-3.222', '-1.924', '-60.801', '-59.16'],
  'category': 'boundary',
  'display_name': 'Manaus, Região Geográfica Imediata de Manaus, Região '
                  'Geográfica Intermediária de Manaus, Amazonas, Região Norte, '
                  '69000-000, Brasil',
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png',
  'importance': 0.6822901469283741,
  'lat': '-3.1316333',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
             'https://osm.org/copyright',
  'lon': '-59.9825041',
  'osm_id': 332493,
  'osm_type': 'relation',
  'place_id': 297367104,
  'place_rank': 16,
  'type': 'administrative'},
 {'boundingbox': ['-3.222', '-1.924', '-60.801', '-59.16'],
  'category': 'boundary',
  'display_name': 'Manaus, Região Geográfica Imediata de Manaus, Região '
                  'Geográfica Intermediária de Manaus, Amazonas, Região Norte, '
                  '69000-000, Brasil',
  'icon': 'https://nominatim.openstr

Normalmente, a decodificação JSON criará dicts ou listas a partir dos dados fornecidos. Se você deseja criar diferentes tipos de objetos, forneça o *object_pairs_hook* ou *object_hook* para `json.loads()`. Por exemplo, aqui está como você decodificaria dados JSON, preservando sua ordem em um `OrderedDict`:

In [17]:
s = '{"name": "ACME", "shares": 50, "price": 490.1}'
from collections import OrderedDict
data = json.loads(s, object_pairs_hook=OrderedDict)
data

OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])

Aqui está como você pode transformar um dicionário JSON em um objeto Python:

In [18]:
class JSONObject:
    def __init__(self, d):
        self.__dict__ = d

In [19]:
data = json.loads(s, object_hook=JSONObject)

In [20]:
 data.name

'ACME'

In [21]:
 data.shares

50

In [22]:
data.price

490.1

Neste último exemplo, o dicionário criado pela decodificação dos dados JSON é passado como um único argumento para `__init__()`. A partir daí, você está livre para usá-lo como quiser, como usá-lo diretamente como o dicionário de instância do objeto.
Existem algumas opções que podem ser úteis para codificar JSON. Se você quiser que a saída seja bem formatada, você pode usar o argumento `indent` para `json.dumps()`. Isso faz com que a saída seja bem impressa em um formato semelhante ao da função `pprint()`. Por exemplo:

In [23]:
data = {
    'name' : 'ACME',
    'shares' : 100,
    'price' : 542.23
}

print(json.dumps(data))

{"name": "ACME", "shares": 100, "price": 542.23}


In [24]:
print(json.dumps(data, indent=4))

{
    "name": "ACME",
    "shares": 100,
    "price": 542.23
}


Se você quiser que as chaves sejam classificadas na saída, use o argumento `sort_keys`:

In [25]:
print(json.dumps(data, sort_keys=True))

{"name": "ACME", "price": 542.23, "shares": 100}


As instâncias normalmente não são serializáveis como JSON. Por exemplo:

In [26]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
p = Point(2, 3)

~~~python
json.dumps(p)
Traceback (most recent call last):
 File "<stdin>", line 1, in <module>
 File "/usr/local/lib/python3.3/json/__init__.py", line 226, in dumps
 return _default_encoder.encode(obj)
 File "/usr/local/lib/python3.3/json/encoder.py", line 187, in encode
 chunks = self.iterencode(o, _one_shot=True)
 File "/usr/local/lib/python3.3/json/encoder.py", line 245, in iterencode
 return _iterencode(o, 0)
 File "/usr/local/lib/python3.3/json/encoder.py", line 169, in default
 raise TypeError(repr(o) + " is not JSON serializable")
TypeError: <__main__.Point object at 0x1006f2650> is not JSON serializable
>>>
~~~

Se você deseja serializar instâncias, pode fornecer uma função que recebe uma instância como entrada e retorna um dicionário que pode ser serializado. Por exemplo:

In [27]:
def serialize_instance(obj):
    d = { '__classname__' : type(obj).__name__ }
    d.update(vars(obj))
    return d

Se você quiser obter uma instância de volta, você pode escrever um código como este:

In [28]:
# Dictionary mapping names to known classes
classes = {
    'Point' : Point
}

def unserialize_object(d):
    clsname = d.pop('__classname__', None)
    if clsname:
        cls = classes[clsname]
        obj = cls.__new__(cls) # Make instance without calling __init__
        for key, value in d.items():
            setattr(obj, key, value)
            return obj
    else:
        return d


Aqui está um exemplo de como essas funções são usadas:

In [29]:
p = Point(2,3)
s = json.dumps(p, default=serialize_instance)
s

'{"__classname__": "Point", "x": 2, "y": 3}'

In [30]:
a = json.loads(s, object_hook=unserialize_object)
a

In [31]:
a.x

2

O módulo json tem uma variedade de outras opções para controlar a interpretação de números de baixo nível, valores especiais como NaN e muito mais. Consulte a documentação para mais detalhes.

## 6.3. Analisando dados XML simples

**Problema**

Você gostaria de extrair dados de um documento XML simples.

**Solução**
O módulo `xml.etree.ElementTree` pode ser usado para extrair dados de documentos XML simples. Para ilustrar, suponha que você queira analisar e fazer um resumo do feed RSS no Planet Python. Aqui está um script que fará isso:

In [32]:
from urllib.request import urlopen
from xml.etree.ElementTree import parse
# Download the RSS feed and parse it
u = urlopen('http://planet.python.org/rss20.xml')
doc = parse(u)

In [33]:
# Extract and output tags of interest
for item in doc.iterfind('channel/item'):
    title = item.findtext('title')
    date = item.findtext('pubDate')
    link = item.findtext('link')
    print(title)
    print(date)
    print(link)
    print()

Real Python: Refactoring: Prepare Your Code to Get Help
Tue, 08 Nov 2022 14:00:00 +0000
https://realpython.com/courses/refactoring-code-to-get-help/

PyCharm: Support Python with JetBrains and PyCharm
Tue, 08 Nov 2022 13:02:00 +0000
https://blog.jetbrains.com/pycharm/2022/11/support-python-with-jetbrains-and-pycharm/

James Bennett: A Python 3.11 "gotcha"
Tue, 08 Nov 2022 01:12:40 +0000
https://www.b-list.org/weblog/2022/nov/08/python-311-gotcha/

Matthew Wright: Matching data between data sources with Python
Mon, 07 Nov 2022 17:55:52 +0000
https://www.wrighters.io/matching-data-between-data-sources-with-python/

PyBites: New on our platform: Typer learning path
Mon, 07 Nov 2022 17:06:33 +0000
https://pybit.es/articles/new-on-our-platform-typer-learning-path/

PyCharm: Webinar: “Speech-to-image generation using Jina”
Mon, 07 Nov 2022 15:13:00 +0000
https://blog.jetbrains.com/pycharm/2022/11/webinar-speech-to-image-generation-using-jina/

Real Python: Python News: What's New From Octobe

Obviamente, se você quiser fazer mais processamento, precisará substituir as instruções `print()` por algo mais interessante.

**Discussão**

Trabalhar com dados codificados como XML é comum em muitos aplicativos. O XML não é apenas amplamente usado como formato para troca de dados na Internet, mas também é um formato comum para armazenar dados de aplicativos (por exemplo, processamento de texto, bibliotecas de música etc.). A discussão a seguir já pressupõe que o leitor esteja familiarizado com os fundamentos do XML.

Em muitos casos, quando o XML está simplesmente sendo usado para armazenar dados, a estrutura do documento é compacta e direta. Por exemplo, o feed RSS do exemplo é semelhante ao seguinte:

~~~html
<?xml version="1.0"?>
 <rss version="2.0" xmlns:dc="http://purl.org/dc/elements/1.1/">
 <channel>
     <title>Planet Python</title>
     <link>http://planet.python.org/</link>
     <language>en</language>
     <description>Planet Python - http://planet.python.org/</description>
     <item>
         <title>Steve Holden: Python for Data Analysis</title>
         <guid>http://holdenweb.blogspot.com/...-data-analysis.html</guid>
         <link>http://holdenweb.blogspot.com/...-data-analysis.html</link>
     <description>...</description>
     <pubDate>Mon, 19 Nov 2012 02:13:51 +0000</pubDate>
     </item>
<item>
    <title>Vasudev Ram: The Python Data model (for v2 and v3)</title>
    <guid>http://jugad2.blogspot.com/...-data-model.html</guid>
    <link>http://jugad2.blogspot.com/...-data-model.html</link>
<description>...</description>
<pubDate>Sun, 18 Nov 2012 22:06:47 +0000</pubDate>
</item>
<item>
    <title>Python Diary: Been playing around with Object Databases</title>
    <guid>http://www.pythondiary.com/...-object-databases.html</guid>
    <link>http://www.pythondiary.com/...-object-databases.html</link>
<description>...</description>
<pubDate>Sun, 18 Nov 2012 20:40:29 +0000</pubDate>
</item>
...
 </channel>
</rss>
~~~

A função `xml.etree.ElementTree.parse()` analisa todo o documento XML em um objeto de documento. A partir daí, você usa métodos como `find()`, `iterfind()` e `findtext()` para pesquisar elementos XML específicos. Os argumentos para essas funções são os nomes de uma tag específica, como canal/item ou título.

Ao especificar tags, você precisa levar em consideração a estrutura geral do documento. Cada operação de localização ocorre em relação a um elemento inicial. Da mesma forma, o tagname que você fornece para cada operação também é relativo ao início. No exemplo, a chamada para `doc.iterfind('canal/item')` procura todos os elementos “item” sob um elemento “canal”. doc representa a parte superior do documento (o elemento “rss” de nível superior). As últimas chamadas para `item.findtext()` ocorrem em relação aos elementos “item” encontrados.

Cada elemento representado pelo módulo `ElementTree` tem alguns atributos e métodos essenciais que são úteis ao analisar. O atributo tag contém o nome da tag, o atributo text contém texto incluído e o método `get()` pode ser usado para extrair atributos (se houver). Por exemplo:

In [34]:
doc

In [36]:
e = doc.find('channel/title')
e

<Element 'title' at 0x000001C96C2DECC0>

In [37]:
e.tag

'title'

In [38]:
e.text

'Planet Python'

In [39]:
e.get('some_attribute')

Deve-se notar que `xml.etree.ElementTree` não é a única opção para análise XML. Para aplicativos mais avançados, você pode considerar `lxml`. Ele usa a mesma interface de programação do `ElementTree`, então o exemplo mostrado nesta receita funciona da mesma maneira. Você simplesmente precisa alterar a primeira importação `from lxml.etree import parse.lxml` oferece o benefício de ser totalmente compatível com os padrões XML. Também é extremamente **rápido** e oferece suporte para recursos como validação, XSLT e XPath.

## 6.4. Analisando arquivos XML enormes de forma incremental

**Problema**

Você precisa extrair dados de um documento XML enorme usando o mínimo de memória possível.

**Solução**

Sempre que você se deparar com o problema do processamento incremental de dados, você deve pensar em iteradores e geradores. Aqui está uma função simples que pode ser usada para processar de forma incremental arquivos XML enormes usando uma pegada de memória muito pequena:

In [40]:
from xml.etree.ElementTree import iterparse
def parse_and_remove(filename, path):
    path_parts = path.split('/')
    doc = iterparse(filename, ('start', 'end'))
    # Skip the root element
    next(doc)
    tag_stack = []
    elem_stack = []
    for event, elem in doc:
        if event == 'start':
            tag_stack.append(elem.tag)
            elem_stack.append(elem)
        elif event == 'end':
            if tag_stack == path_parts:
                yield elem
                elem_stack[-2].remove(elem)
            try:
                tag_stack.pop()
                elem_stack.pop()
            except IndexError:
                pass

Para testar a função, agora você precisa encontrar um arquivo XML grande para trabalhar. Muitas vezes você pode encontrar esses arquivos em sites governamentais e de dados abertos. Por exemplo, você pode baixar o banco de dados de buracos de Chicago como XML. No momento da redação deste artigo, o arquivo baixado consiste em mais de 100.000 linhas de dados, que são codificados assim:

Suponha que você queira escrever um script que classifique os CEPs pelo número de relatórios de buracos. Para fazer isso, você poderia escrever um código como este:

In [47]:
from xml.etree.ElementTree import parse
from collections import Counter
potholes_by_zip = Counter()
doc = parse('potholes1.xml')
for pothole in doc.iterfind('row/row'):
    potholes_by_zip[pothole.findtext('zip')] += 1
for zipcode, num in potholes_by_zip.most_common():
    print(zipcode, num)

60629 22554
60617 19126
60628 19034
60618 18630
60634 18244
60647 17758
60620 17743
60643 17486
60619 16867
60632 16753
60638 16351
60639 15810
60630 15380
60608 15049
60609 13858
60641 13526
60623 12973
60625 12678
60614 12527
60631 12406
60646 10908
60652 10172
60651 10000
60645 9804
60622 9572
60644 9381
60649 9260
60659 8975
60612 8871
60636 8806
60657 8782
60626 8666
60637 8619
60616 7845
60660 7819
60655 7651
60640 7516
60624 7063
60656 6834
60607 6579
60621 6185
60642 6142
60615 5322
60613 5100
60611 4942
60610 4616
60707 4460
60653 4174
60605 3215
60601 3061
60654 2981
60633 2932
None 2318
60661 1420
60603 1177
60604 1022
60606 1008
60827 988
0 719
60602 706
60666 73
60635 27
60627 11
60687 2
60018 1


Esta versão do código é executada com uma pegada de memória de apenas 7 MB—uma enorme economia!

**Discussão**

Esta receita se baseia em dois recursos principais do módulo ElementTree. Primeiro, o método iterparse() permite o processamento incremental de documentos XML. Para usá-lo, você fornece o nome do arquivo junto com uma lista de eventos que consiste em um ou mais dos seguintes:**start**, **end**, **start-ns** e **end-ns**. O iterador criado por `iterparse()` produz tuplas no form(event, elem), onde event é um dos eventos listados e elem é o elemento XML resultante. Por exemplo:

In [48]:
data = iterparse('potholes.xml',('start','end'))

In [49]:
next(data)

('start', <Element 'response' at 0x000001C96C643E50>)

In [50]:
next(data)

('start', <Element 'row' at 0x000001C96D091040>)

In [51]:
next(data)

('start', <Element 'row' at 0x000001C96D0763B0>)

In [52]:
next(data)

('start', <Element 'creation_date' at 0x000001C96D0760E0>)

In [53]:
next(data)

('end', <Element 'creation_date' at 0x000001C96D0760E0>)

In [54]:
next(data)

('start', <Element 'status' at 0x000001C96D076220>)

In [55]:
next(data)

('end', <Element 'status' at 0x000001C96D076220>)

Os eventos **start** são criados quando um elemento é criado pela primeira vez, mas ainda não é preenchido com nenhum outro dado (por exemplo, elementos filho). eventos **end** são criados quando um elemento é concluído.

Embora não sejam mostrados nesta receita, os eventos **start-ns** e **end-ns** são usados para manipular declarações de namespace XML.

Nesta receita, os eventos de **start** e **end** são usados para gerenciar pilhas de elementos e tags. As pilhas representam a estrutura hierárquica atual do documento conforme ele está sendo analisado e também são usadas para determinar se um elemento corresponde ao caminho solicitado fornecido à função **parse_and_remove()** . Se uma correspondência for feita, o rendimento é usado para emiti-lo de volta para o chamador.

A seguinte declaração após o **yield** é o recurso principal do **ElementTree** que faz com que essa receita economize memória:
~~~python
elem_stack[-2].remove(elem)
~~~

Essa instrução faz com que o elemento gerado anteriormente seja removido de seu pai.Supondo que nenhuma referência seja deixada a ele em nenhum outro lugar, o elemento é destruído e a memória recuperada.

O efeito **end** da análise iterativa e a remoção de nós é uma varredura incremental altamente eficiente sobre o documento. Em nenhum momento uma árvore de documentos completa é construída. No entanto, ainda é possível escrever código que processe os dados XML de maneira direta.

A principal desvantagem dessa receita é seu desempenho em tempo de execução. Quando testada, a versão do código que lê o documento inteiro na memória primeiro é executada aproximadamente duas vezes mais rápido que a versão que o processa de forma incremental. No entanto, requer mais de 60 vezes mais memória. Portanto, se o uso de memória é uma preocupação maior, a versão incremental é uma grande vitória.